## Load and Explore the Dataset

In [5]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('../data/df_file.csv')  

# Map numeric labels to category names
label_to_category = {
    0: 'Politics',
    1: 'Sport',
    2: 'Technology',
    3: 'Entertainment',
    4: 'Business'
}

# Add a new category called 'Other' for unmatched texts
label_to_category[5] = 'Other'

# Reverse mapping for category to label (useful for prediction)
category_to_label = {v: k for k, v in label_to_category.items()}

# Add a new column for category names
df['category'] = df['Label'].map(label_to_category)

# Display basic info
print("Dataset Overview:")
print(df.head())

print("\nDataset Shape:", df.shape)
print("\nColumn Names and Data Types:")
print(df.dtypes)

# Display class distribution with category names
print("\nClass Distribution:")
print(df['category'].value_counts())

Dataset Overview:
                                                Text  Label  category
0  Budget to set scene for election\n \n Gordon B...      0  Politics
1  Army chiefs in regiments decision\n \n Militar...      0  Politics
2  Howard denies split over ID cards\n \n Michael...      0  Politics
3  Observers to monitor UK election\n \n Minister...      0  Politics
4  Kilroy names election seat target\n \n Ex-chat...      0  Politics

Dataset Shape: (2225, 3)

Column Names and Data Types:
Text        object
Label        int64
category    object
dtype: object

Class Distribution:
category
Sport            511
Business         510
Politics         417
Technology       401
Entertainment    386
Name: count, dtype: int64
